In [1]:
import pandas as pd
import json
import requests
import numpy as np
import pickle

In [2]:
game_data = []
year = '/2022'
season_type = '02'

In [42]:
base = 'https://statsapi.web.nhl.com/api/v1/'
results = requests.get('https://statsapi.web.nhl.com/api/v1/teams')

In [11]:
teams = results.json()

In [81]:
rosterDict = {}

for team  in  teams['teams']:
    print(team['name'], team['id'])
    results = requests.get('https://statsapi.web.nhl.com/api/v1/teams/'+ str(team['id']) + '/roster')
    data = results.json()
    rosterDict[team['id']] =  data['roster']

New Jersey Devils 1
New York Islanders 2
New York Rangers 3
Philadelphia Flyers 4
Pittsburgh Penguins 5
Boston Bruins 6
Buffalo Sabres 7
Montréal Canadiens 8
Ottawa Senators 9
Toronto Maple Leafs 10
Carolina Hurricanes 12
Florida Panthers 13
Tampa Bay Lightning 14
Washington Capitals 15
Chicago Blackhawks 16
Detroit Red Wings 17
Nashville Predators 18
St. Louis Blues 19
Calgary Flames 20
Colorado Avalanche 21
Edmonton Oilers 22
Vancouver Canucks 23
Anaheim Ducks 24
Dallas Stars 25
Los Angeles Kings 26
San Jose Sharks 28
Columbus Blue Jackets 29
Minnesota Wild 30
Winnipeg Jets 52
Arizona Coyotes 53
Vegas Golden Knights 54
Seattle Kraken 55


In [137]:
bioData = pd.DataFrame()
for team in teams['teams']:
    teamId = team['id']
    for player in rosterDict[teamId]:
        playerId = player['person']['id']
        response = requests.get(base+'people/'+str(playerId))
        bio = response.json()
        data = pd.json_normalize(bio['people'])
        bioData = bioData.append(data)

In [195]:
endUrl = '/stats?stats=statsSingleSeason&season=20212022'
stats2022 = pd.DataFrame()
for player in bioData['id']:
    playerId = player
    response = requests.get(base+'people/'+str(playerId)+endUrl)
    stats = response.json()
    for stat in stats['stats']:
        row = pd.json_normalize(stat['splits'])
        row.insert(0,'id',playerId)
        stats2022 = stats2022.append(row)

In [196]:
stats2022.head()

,id,season,stat.timeOnIce,stat.ot,stat.shutouts,stat.ties,stat.wins,stat.losses,stat.saves,stat.powerPlaySaves,...,stat.shortHandedGoals,stat.shortHandedPoints,stat.shortHandedTimeOnIce,stat.blocked,stat.plusMinus,stat.points,stat.shifts,stat.evenTimeOnIcePerGame,stat.shortHandedTimeOnIcePerGame,stat.powerPlayTimeOnIcePerGame
0,8473541,20212022,510:06,1.0,0.0,0.0,4.0,4.0,238.0,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8477425,20212022,44:17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,00:32,1.0,-2.0,0.0,59.0,13:18,00:10,01:16
0,8477970,20212022,2317:23,6.0,4.0,0.0,20.0,12.0,1014.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8479407,20212022,1324:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,03:31,30.0,0.0,73.0,1749.0,14:39,00:02,02:43
0,8480003,20212022,735:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,09:20,16.0,-9.0,23.0,1087.0,12:28,00:10,00:29


In [197]:
stats2022.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 773 entries, 0 to 0
Data columns (total 49 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                773 non-null    int64  
 1   season                            773 non-null    object 
 2   stat.timeOnIce                    773 non-null    object 
 3   stat.ot                           82 non-null     float64
 4   stat.shutouts                     82 non-null     float64
 5   stat.ties                         82 non-null     float64
 6   stat.wins                         82 non-null     float64
 7   stat.losses                       82 non-null     float64
 8   stat.saves                        82 non-null     float64
 9   stat.powerPlaySaves               82 non-null     float64
 10  stat.shortHandedSaves             82 non-null     float64
 11  stat.evenSaves                    82 non-null     float64
 12  stat.short

In [198]:
stats2022.to_csv('stats2022NHL.csv')
bioData.to_csv('bioDataNHL.csv')